## Imports and Setup

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json

# Qiskit
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
import qiskit_aer.noise as noise
from mitiq import zne

# Our clean utilities
from utils.quantum_execution import (
    calibrate_mthree
)
from utils.experiment_runners import (
    run_baseline_experiment, run_pcs_experiment, 
    run_ideal_experiment, extrapolate_pcs_data,
    collect_zne_data, extrapolate_zne_data
)
from utils.circuit_generation import generate_rand_mirror_cliff, ghz_mirror_circ
from utils.layout_finder import (
    find_best_payload_layout, build_pcs_config
)

print("✅ All imports successful!")

## Configuration

In [ ]:
# Experiment parameters
USE_REAL_HARDWARE = False
BACKEND_NAME = "ibm_kingston"

num_qubits = 4
max_num_checks = 3
circ_depth = 5
num_circuits = 5

barriers = True

# Shot allocation
pcs_shots_per_check = 20_000
total_shots = pcs_shots_per_check * max_num_checks
baseline_shots = pcs_shots_per_check

# Error mitigation settings
enable_mthree = True
enable_twirling = True

# Observable
pauli_string = "Z" * num_qubits

# PCE/ZNE methods
pce_methods = ["linear", "exponential"]
zne_methods = ["linear", "richardson"]
scale_factors = [1, 3, 5]

print(f"Configuration:")
print(f"  Backend: {BACKEND_NAME} ({'real' if USE_REAL_HARDWARE else 'simulated'})")
print(f"  Circuits: {num_circuits} × {num_qubits}q depth-{circ_depth}")
print(f"  Shots: {total_shots} total, {pcs_shots_per_check} per PCS check")
print(f"  Observable: {pauli_string}")

## Backend Setup

In [3]:
from qiskit_ibm_runtime.fake_provider import *
from qiskit.visualization import plot_error_map

# Initialize backend
service = QiskitRuntimeService()

if USE_REAL_HARDWARE:
    backend = service.backend(BACKEND_NAME)
    print(f"Using real hardware: {backend.name}")
else:
    if BACKEND_NAME == 'ibm_fez':
        backend = FakeFez()

    elif BACKEND_NAME == 'ibm_brisbane':
        backend = FakeBrisbane()

    else:
        backend = None
        print(f"Warning: No fake backend found for '{BACKEND_NAME}'")
    
    
# plot_error_map(backend, figsize=(12, 10))

## Generate Test Circuits

In [ ]:
# Generate random mirrored Clifford circuits
print("Generating random mirrored Clifford circuits...")
circuits = [generate_rand_mirror_cliff(num_qubits, circ_depth) for _ in range(num_circuits)]

print(f"Generated {len(circuits)} circuits")

# Show example circuit
print("\nExample circuit:")
display(circuits[0].draw('mpl', fold=-1))

## Find Optimal Layouts

In [ ]:
# Find optimal payload layouts for all circuits
print("Finding optimal payload layouts...")
circuit_configs = []

for circ_idx, circuit in enumerate(circuits):
    print(f"\nCircuit {circ_idx + 1}/{num_circuits}:")
    
    # Find the best payload layout that can support our max number of checks
    try:
        payload_layout = find_best_payload_layout(circuit, backend, max_num_checks, verbose=True)
        print(f"  ✅ Payload layout: {payload_layout}")
        
        circuit_configs.append({
            'circuit': circuit,
            'payload_layout': payload_layout
        })
        
    except RuntimeError as e:
        raise RuntimeError(f"❌ Circuit {circ_idx + 1}: {e}")

print(f"\n✅ Found layouts for all {len(circuits)} circuits")

In [ ]:
circuit_configs[0]["payload_layout"]

## Calibrate Readout Mitigation

In [ ]:
# Calculate all qubits that will be used for mthree calibration
all_qubits_used = set()

for config in circuit_configs:
    payload_layout = config['payload_layout']
    all_qubits_used.update(payload_layout)
    
    # Add potential ancilla positions for max checks
    try:
        from utils.layout_finder import get_check_assignments
        assignments = get_check_assignments(payload_layout, backend, max_num_checks)
        ancilla_positions = [pair[1] for pair in assignments]
        all_qubits_used.update(ancilla_positions)
    except Exception as e:
        print(f"Warning: Could not determine ancilla positions: {e}")

print(f"Total qubits to be used: {len(all_qubits_used)} - {sorted(all_qubits_used)}")

# Calibrate mthree if using real hardware
mthree_mit = None

if enable_mthree:
    print(f"Calibrating mthree for {len(all_qubits_used)} qubits...")
    mthree_mit = calibrate_mthree(backend, list(all_qubits_used))
    print("✅ mthree calibration complete")
else:
    print("mthree disabled")

## Main Experiment Loop

In [ ]:
# Results storage
all_results = []

# Create save directory
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_dir = f"results/{'real' if USE_REAL_HARDWARE else 'sim'}_{backend.backend_name}"
os.makedirs(results_dir, exist_ok=True)
results_file = f"{results_dir}/pce_vs_zne_{num_qubits}q_{circ_depth}d.jsonl"

print(f"Results will be saved to: {results_file}\n")

for circuit_idx, circuit in enumerate(circuits):
    print(f"\n{'='*60}")
    print(f"CIRCUIT {circuit_idx + 1}/{len(circuits)}")
    print(f"{'='*60}")
    
    # Get pre-computed payload layout
    config = circuit_configs[circuit_idx]
    payload_layout = config['payload_layout']
    print(f"Payload Layout: {payload_layout}")
    print("payload circuit:")
    display(circuit.draw("mpl", fold=-1))
    
    # 1. IDEAL EXPECTATION VALUE
    print("\n1. Computing ideal expectation value...")
    ideal_expectation = run_ideal_experiment(circuit=circuit, pauli_string=pauli_string, shots=baseline_shots)
    print(f"   Ideal: {ideal_expectation:.6f}")
    
    # 2. BASELINE EXPERIMENT
    print("\n2. Running baseline experiment...")
    baseline_exp, baseline_meta = run_baseline_experiment(
        circuit, backend, pauli_string, baseline_shots,
        layout=payload_layout, twirling=False, mthree=None
    )
    print(f"   Baseline: {baseline_exp:.6f}")
    
    # 3. PCS EXPERIMENTS - build configurations on-demand
    print("\n3. Running PCS experiments...")
    pcs_results = []
    pcs_metadata = []
    
    for num_checks in range(1, max_num_checks + 1):
        print(f"   PCS with {num_checks} checks...")
        
        # Build PCS configuration on-demand
        pcs_config = build_pcs_config(circuit, payload_layout, num_checks, backend)
        
        # Run PCS experiment
        signs = ["+1"] * num_checks  # Mirror circuits have positive checks
        pcs_exp, pcs_meta = run_pcs_experiment(
            pcs_config['circuit'], backend, pauli_string, pcs_shots_per_check,
            num_checks, signs, layout=pcs_config['layout'],
            twirling=enable_twirling, mthree=mthree_mit
        )
        
        pcs_results.append(pcs_exp)
        pcs_metadata.append(pcs_meta)
        
        print(f"      Score: {pcs_exp:.6f}, Post-selection: {pcs_meta['post_selection_rate']:.2%}")
    
    # 4. PCE EXTRAPOLATION
    print("\n4. Applying PCE extrapolation...")
    pce_results = {}
    
    if max_num_checks >= 2:
        for method in pce_methods:
            for num_fit_points in range(2, max_num_checks + 1):
                try:
                    pce_exp, pce_meta = extrapolate_pcs_data(
                        pcs_results, num_fit_points, num_qubits,
                        method=method, show_plot=True
                    )
                    
                    key = f"pce_{method}_{num_fit_points}pts"
                    pce_results[key] = pce_exp
                    print(f"   {key}: {pce_exp:.6f}")
                    
                except Exception as e:
                    print(f"   {method} extrapolation failed: {e}")
                    pce_results[f"pce_{method}_{num_fit_points}pts"] = None
    
    # 5. ZNE DATA COLLECTION (Execute circuits once)
    print("\n5. Collecting ZNE data...")
    zne_shots_per_scale = total_shots // len(scale_factors)
    print("zne shots per scale = ", zne_shots_per_scale)
    
    zne_expectation_values, zne_data_meta = collect_zne_data(
        circuit, backend, pauli_string, zne_shots_per_scale,
        scale_factors, zne.scaling.fold_global,
        layout=payload_layout, twirling=enable_twirling, mthree=mthree_mit
    )
    
    # 6. ZNE EXTRAPOLATION (Apply multiple methods to same data)
    print("\n6. Applying ZNE extrapolation methods...")
    zne_results = []
    
    for method in zne_methods:
        # Skip Richardson with non-integer scales
        if method == "richardson" and any(not isinstance(s, int) for s in scale_factors):
            print(f"   Skipping {method} (requires integer scales)")
            continue
            
        try:
            zne_exp, zne_meta = extrapolate_zne_data(
                zne_expectation_values, scale_factors, 
                method=method, show_plot=True
            )
            
            zne_results.append({**zne_meta, 'expectation': zne_exp})
            print(f"   ZNE {method}: {zne_exp:.6f}")
            
        except Exception as e:
            print(f"   ZNE {method} failed: {e}")
    
    # 7. SAVE RESULTS
    circuit_results = {
        'circuit_idx': circuit_idx,
        'timestamp': datetime.now().isoformat(),
        'layout': payload_layout,
        'ideal': ideal_expectation,
        'baseline': baseline_exp,
        'pcs': pcs_results,
        'pcs_metadata': pcs_metadata,
        'pce': pce_results,
        'zne_data': zne_expectation_values,
        'zne_scale_factors': scale_factors,
        'zne': zne_results,
        'config': {
            'num_qubits': num_qubits,
            'circ_depth': circ_depth,
            'backend': backend.backend_name,
            'pauli_string': pauli_string,
            'mthree_enabled': enable_mthree,
            'twirling_enabled': enable_twirling
        }
    }
    
    # Save incrementally
    with open(results_file, 'a') as f:
        json.dump(circuit_results, f, default=str)
        f.write('\n')
    
    all_results.append(circuit_results)
    
    print(f"\n✅ Circuit {circuit_idx + 1} complete and saved")

print(f"\n{'='*60}")
print("🎉 ALL EXPERIMENTS COMPLETE!")
print(f"Results saved to: {results_file}")
print(f"{'='*60}")

## Results Analysis

In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

# num_qubits = 4
# circ_depth = 20

num_qubits = 4
circ_depth = 25

results_dir = f"results/real_ibm_kingston"
results_file = f"{results_dir}/pce_vs_zne_{num_qubits}q_{circ_depth}d.jsonl"

In [ ]:
# Optional: Detailed per-circuit breakdown
# This cell provides a detailed view of individual circuit results

def load_results(filename):
    """Load incrementally saved results from JSONL file."""
    results = []
    with open(filename, 'r') as f:
        for line in f:
            results.append(json.loads(line))
    return results

def show_detailed_breakdown(results_file):
    """Show detailed breakdown of results for each circuit."""
    try:
        loaded_results = load_results(results_file)
        print(f"📊 DETAILED BREAKDOWN - {len(loaded_results)} circuits")
        print("="*60)

        for i, result in enumerate(loaded_results):
            print(f"\n🔸 Circuit {i+1}:")
            print(f"   Layout: {result['layout']}")
            print(f"   Ideal: {result['ideal']:.4f}")
            print(f"   Baseline: {result['baseline']:.4f}")
            
            if result['pcs']:
                print(f"   PCS results: {[f'{val:.4f}' for val in result['pcs']]}")
                print(f"   Best PCS: {max(result['pcs']):.4f}")
            
            valid_pce = {k: v for k, v in result['pce'].items() if v is not None}
            if valid_pce:
                best_pce_method = max(valid_pce, key=valid_pce.get)
                print(f"   Best PCE: {valid_pce[best_pce_method]:.4f} ({best_pce_method})")
            
            if result['zne']:
                best_zne = max(result['zne'], key=lambda x: x['expectation'])
                print(f"   Best ZNE: {best_zne['expectation']:.4f} ({best_zne['zne_method']})")
                
            # Show post-selection rates if available
            if 'pcs_metadata' in result and result['pcs_metadata']:
                post_rates = [meta['post_selection_rate'] for meta in result['pcs_metadata']]
                print(f"   Post-selection rates: {[f'{rate:.2%}' for rate in post_rates]}")

    except FileNotFoundError:
        print(f"❌ Results file {results_file} not found. Run the experiments first.")

# Uncomment the line below to show detailed breakdown
show_detailed_breakdown(results_file)

In [ ]:
# Set publication-quality parameters
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']
rcParams['font.size'] = 11
rcParams['axes.labelsize'] = 25
rcParams['xtick.labelsize'] = 10
rcParams['ytick.labelsize'] = 10
rcParams['legend.fontsize'] = 10
rcParams['figure.dpi'] = 100
rcParams['savefig.dpi'] = 300

def load_results(filename):
    """Load incrementally saved results from JSONL file."""
    results = []
    with open(filename, 'r') as f:
        for line in f:
            results.append(json.loads(line))
    return results

def collect_method_values(results):
    """Collect all values for each method from results."""
    method_values = {
        'ideal': [r['ideal'] for r in results],
        'baseline': [r['baseline'] for r in results]
    }

    # PCE results
    for result in results:
        for key, value in result['pce'].items():
            if value is not None:
                if key not in method_values:
                    method_values[key] = []
                method_values[key].append(value)

    # ZNE results
    for result in results:
        for zne_result in result['zne']:
            key = f"zne_{zne_result['zne_method']}"
            if key not in method_values:
                method_values[key] = []
            method_values[key].append(zne_result['expectation'])

    return method_values

def get_clean_label(method_name):
    """Convert method name to clean label for plotting with line breaks."""
    if method_name == 'baseline':
        return 'Baseline'

    # Handle PCE methods - extract extrapolation and check count
    if method_name.startswith('pce_'):
        parts = method_name.split('_')
        if len(parts) >= 3:
            extrapolation = parts[1]  # 'linear' or 'exponential'
            check_info = parts[2]      # e.g., '2pts' or '3pts'

            # Extract number from check_info (e.g., '2pts' -> '2')
            num_checks = ''.join(filter(str.isdigit, check_info))

            # Create two-line label with full method name
            if extrapolation == 'linear':
                return f'PCE-linear\n({num_checks} checks)'
            elif extrapolation == 'exponential':
                return f'PCE-exponential\n({num_checks} checks)'
            else:
                return f'PCE-{extrapolation}\n({num_checks} checks)'
        return method_name

    # Handle ZNE methods
    if method_name.startswith('zne_'):
        zne_type = method_name.replace('zne_', '')
        if 'linear' in zne_type:
            return 'ZNE-linear'
        elif 'richardson' in zne_type:
            return 'ZNE-richardson'
        elif 'exponential' in zne_type:
            return 'ZNE-exponential'
        elif 'polynomial' in zne_type:
            return 'ZNE-polynomial'
        return f'ZNE-{zne_type}'

    return method_name

def get_method_color(method_name):
    """Get standard scientific color scheme for each method type."""
    if method_name == 'baseline':
        return 'tab:green'
    elif method_name.startswith('pce_'):
        return 'tab:orange'
    elif method_name.startswith('zne_'):
        return 'tab:blue'
    else:
        return '#808080'  # Gray for other

def plot_expectation_values(results_file, num_qubits, circ_depth):
    """Create publication-ready plot of expectation values."""

    # Load results
    try:
        results = load_results(results_file)
        print(f"Loaded {len(results)} circuit results")
    except FileNotFoundError:
        print(f"Results file {results_file} not found")
        return

    if not results:
        print("No results to visualize.")
        return

    # Collect method values
    method_values = collect_method_values(results)

    # Calculate reference values
    ideal_mean = np.mean(method_values['ideal'])
    baseline_mean = np.mean(method_values['baseline'])

    # Build ordered list of methods to plot: baseline, ZNE, then PCE
    methods_to_plot = ['baseline']
    zne_methods = sorted([k for k in method_values.keys() if k.startswith('zne_') and method_values[k]])
    pce_methods = sorted([k for k in method_values.keys() if k.startswith('pce_') and method_values[k]])
    methods_to_plot.extend(zne_methods)
    methods_to_plot.extend(pce_methods)

    # Calculate statistics
    # NOTE: Error bars show ±1 standard deviation (not confidence interval)
    # This represents ~68% of the data if normally distributed
    means = [np.mean(method_values[m]) for m in methods_to_plot]
    stds = [np.std(method_values[m]) if len(method_values[m]) > 1 else 0 for m in methods_to_plot]
    n_samples = [len(method_values[m]) for m in methods_to_plot]

    # Get labels and colors
    labels = [get_clean_label(m) for m in methods_to_plot]
    colors = [get_method_color(m) for m in methods_to_plot]

    # Create plot
    fig, ax = plt.subplots(figsize=(10, 5))

    print(stds)

    # Bar plot with standard deviation error bars
    bars = ax.bar(range(len(methods_to_plot)), means, yerr=stds,
                capsize=4, alpha=0.8, color=colors,
                edgecolor='black', linewidth=0.5)

    # Add value labels on top of bars but shifted right to avoid error bars
    for i, (bar, mean, std) in enumerate(zip(bars, means, stds)):
        # Position text on top of bar, shifted to the right
        x_offset = 0.08  # Shift right to avoid error bar
        x_pos = bar.get_x() + bar.get_width()/2 + x_offset
        y_pos = bar.get_height() + 0.002 * (ax.get_ylim()[1] - ax.get_ylim()[0])  # Just above bar

        ax.text(x_pos, y_pos, f'{mean:.3f}',
                ha='left', va='bottom',  # Left-aligned from offset position
                fontsize=9, fontweight='normal')

    # Reference lines
    ax.axhline(y=ideal_mean, color='tab:red', linestyle='--',
                linewidth=1.5, alpha=0.8)

    # Formatting
    ax.set_xticks(range(len(methods_to_plot)))
    ax.set_xticklabels(labels, rotation=0, ha='center', va='top')
    ax.set_ylabel(r'$\langle Z^{\otimes n} \rangle$', fontsize=12)
    ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.5)
    ax.set_xlim(-0.5, len(methods_to_plot)-0.5 + 0.2)  # Small extra space for labels

    # Adjust y-axis to accommodate value labels and error bars
    y_range = max(means) - min(means)
    y_margin = 0.18 * y_range
    ax.set_ylim(min(means) - max(stds) - y_margin,
                max(means) + max(stds) + y_margin)

    # Adjust bottom margin to accommodate two-line labels
    plt.subplots_adjust(bottom=0.15)

    # Legend
    from matplotlib.patches import Patch
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='tab:red', linestyle='--', linewidth=1.5, label='Ideal'),
        Patch(facecolor='tab:green', alpha=0.8, label='Baseline'),
        Patch(facecolor='tab:blue', alpha=0.8, label='ZNE'),
        Patch(facecolor='tab:orange', alpha=0.8, label='PCE')
    ]
    ax.legend(handles=legend_elements, loc='best', framealpha=0.95)

    plt.tight_layout()

    # Save
    output_base = f"expectation_values_{num_qubits}q_{circ_depth}d"
    plt.savefig(f"{output_base}.pdf", bbox_inches='tight', pad_inches=0.1)
    plt.savefig(f"{output_base}.png", bbox_inches='tight', pad_inches=0.1, dpi=300)
    plt.show()

    print(f"\nFigure saved as {output_base}.pdf and {output_base}.png")

    # Explain error bars clearly
    print("\n" + "="*60)
    print("ERROR BAR INTERPRETATION:")
    print("="*60)
    print("• Error bars show ±1 standard deviation (SD)")
    print("• This is NOT a 95% confidence interval")
    print("• ±1 SD captures ~68% of the data (if normally distributed)")
    print("• ±2 SD would capture ~95% of the data")
    print("• Current error bars show the variability/spread of results")
    print("\nFor 95% confidence intervals, multiply error bars by ~1.96")
    print("Or use standard error: SD/√n for confidence in the mean")
    print("="*60)

    # Print detailed summary
    print(f"\nIdeal: {ideal_mean:.6f}, Baseline: {baseline_mean:.6f}")
    print("\nMethod performance (mean ± std dev):")
    for label, mean_val, std_val, n in zip(labels[1:], means[1:], stds[1:], n_samples[1:]):
        clean_print_label = label.replace('\n', ' ')
        error = abs(mean_val - ideal_mean)
        improvement = (1 - error/abs(baseline_mean - ideal_mean)) * 100
        print(f"  {clean_print_label}:")
        print(f"    Mean: {mean_val:.6f} ± {std_val:.6f} (n={n} circuits)")
        print(f"    Error reduction: {improvement:.1f}%")

# Main execution
plot_expectation_values(results_file, num_qubits, circ_depth)



In [4]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

# ── Publication parameters (1.15× original font sizes) ────────────────
rcParams['font.family']    = 'serif'
rcParams['font.serif']     = ['Times New Roman', 'DejaVu Serif']
rcParams['font.size']      = 20.7    # was 18
rcParams['axes.titlesize'] = 29.9    # was 26
rcParams['axes.labelsize'] = 29.9    # was 26
rcParams['xtick.labelsize']= 20.7    # was 18
rcParams['ytick.labelsize']= 28.75   # was 25
rcParams['legend.fontsize']= 20.7    # was 18
rcParams['figure.dpi']     = 150
rcParams['savefig.dpi']    = 300
rcParams['hatch.linewidth']= 1.2     # crisper hatches

def load_results(filename):
    """Load incrementally saved results from JSONL file."""
    results = []
    with open(filename, 'r') as f:
        for line in f:
            results.append(json.loads(line))
    return results

def collect_method_values(results):
    """Collect all values for each method from results."""
    method_values = {
        'ideal': [r['ideal'] for r in results],
        'baseline': [r['baseline'] for r in results]
    }

    # PCE results
    for result in results:
        for key, value in result['pce'].items():
            if value is not None:
                method_values.setdefault(key, []).append(value)

    # ZNE results
    for result in results:
        for zne_result in result['zne']:
            key = f"zne_{zne_result['zne_method']}"
            method_values.setdefault(key, []).append(zne_result['expectation'])

    return method_values

def get_clean_label(method_name):
    """Convert method name to clean label for plotting with line breaks."""
    if method_name == 'baseline':
        return 'Baseline'

    if method_name.startswith('pce_'):
        parts = method_name.split('_')
        if len(parts) >= 3:
            extrap = parts[1]
            num_checks = ''.join(filter(str.isdigit, parts[2]))
            if extrap == 'linear':
                return f'Linear\n ({num_checks} checks)'
            elif extrap == 'exponential':
                return f'Exponential\n ({num_checks} checks)'
            else:
                return f'PCE-{extrap}\n{num_checks} checks'
        return method_name

    if method_name.startswith('zne_'):
        z = method_name.replace('zne_', '')
        if 'linear' in z:       return 'Linear'
        if 'richardson' in z:   return 'Richardson'
        if 'exponential' in z:  return 'Exponential'
        if 'polynomial' in z:   return 'Polynomial'
        return f'ZNE-{z}'

    return method_name

def get_method_color(method_name):
    """Get standard scientific color scheme for each method type."""
    if method_name == 'baseline':     return 'tab:green'
    if method_name.startswith('pce_'):return 'tab:orange'
    if method_name.startswith('zne_'):return 'tab:blue'
    return '#808080'  # Gray

def get_method_hatch(method_name):
    """Hatch pattern per category for colorblind accessibility."""
    if method_name == 'baseline':      return '.'  # diagonal
    if method_name.startswith('zne_'): return '/'   # cross
    if method_name.startswith('pce_'): return 'x'   # dots
    return ''  # default: no hatch

def plot_expectation_values(results_file, num_qubits, circ_depth):
    """Create publication-ready plot of expectation values."""
    try:
        results = load_results(results_file)
        print(f"Loaded {len(results)} circuit results")
    except FileNotFoundError:
        print(f"Results file {results_file} not found")
        return

    if not results:
        print("No results to visualize.")
        return

    method_values = collect_method_values(results)

    ideal_mean    = np.mean(method_values['ideal'])
    baseline_mean = np.mean(method_values['baseline'])

    methods_to_plot = ['baseline']

    # ---- ZNE methods (unchanged sorting) ----
    zne_methods = sorted([k for k in method_values if k.startswith('zne_') and method_values[k]])

    # ---- PCE methods: enforce order L2, E3, L3, exclude E2 ----
    pce_all = [k for k in method_values if k.startswith('pce_') and method_values[k] 
               and not (k.startswith('pce_exponential') and '2' in k.split('_')[-1])]
    desired = [('linear', 2), ('exponential', 3), ('linear', 3)]
    pce_ordered = []

    for ex, n in desired:
        for k in pce_all:
            parts = k.split('_')
            if len(parts) >= 3:
                extrap = parts[1]
                digits = ''.join(filter(str.isdigit, parts[2]))
                checks = int(digits) if digits else None
                if extrap == ex and checks == n and k not in pce_ordered:
                    pce_ordered.append(k)
                    break
    # Append any remaining PCE keys not matched, still excluding exponential_2
    for k in pce_all:
        if k not in pce_ordered and not (k.startswith('pce_exponential') and '2' in k.split('_')[-1]):
            pce_ordered.append(k)

    methods_to_plot.extend(zne_methods + pce_ordered)

    means = [np.mean(method_values[m]) for m in methods_to_plot]
    stds  = [np.std(method_values[m]) if len(method_values[m]) > 1 else 0 for m in methods_to_plot]
    n_samples = [len(method_values[m]) for m in methods_to_plot]

    labels  = [get_clean_label(m) for m in methods_to_plot]
    colors  = [get_method_color(m) for m in methods_to_plot]
    hatches = [get_method_hatch(m) for m in methods_to_plot]

    fig, ax = plt.subplots(figsize=(12, 6))

    bars = ax.bar(
        range(len(methods_to_plot)), means, yerr=stds,
        capsize=5, alpha=0.85, color=colors,
        edgecolor='black', linewidth=0.9
    )

    # Apply hatches to each bar
    for bar, hatch in zip(bars, hatches):
        bar.set_hatch(hatch)

    # Bar value labels
    for i, (bar, mean, std) in enumerate(zip(bars, means, stds)):
        x_offset = 0.08
        x_pos = bar.get_x() + bar.get_width()/2 + x_offset
        y_pos = bar.get_height() + 0.002 * (ax.get_ylim()[1] - ax.get_ylim()[0])
        ax.text(x_pos, y_pos, f'{mean:.3f}',
                ha='left', va='bottom',
                fontsize=18.4, fontweight='normal')

    # Reference line
    ax.axhline(y=ideal_mean, color='tab:red', linestyle='--', linewidth=1.8, alpha=0.9, label='Ideal')

    # Formatting
    ax.set_xticks(range(len(methods_to_plot)))
    ax.set_xticklabels(labels, rotation=0, ha='center', va='top')
    ax.set_ylabel(r'$\langle Z^{\otimes n} \rangle$', fontsize=23.0)
    ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.8)
    ax.set_xlim(-0.5, len(methods_to_plot) - 0.5)

    # Y-lims with margin
    y_range  = max(means) - min(means)
    y_margin = 0.18 * y_range if y_range > 0 else 0.05
    ax.set_ylim(min(means) - max(stds) - y_margin,
                max(means) + max(stds) + y_margin)

    # Larger tick labels
    ax.tick_params(axis='both', which='major', labelsize=20.7)

    # Legend with matching hatches
    from matplotlib.patches import Patch
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='tab:red', linestyle='--', linewidth=1.8, label='Ideal'),
        Patch(facecolor='tab:green',  edgecolor='black', alpha=0.85, hatch='.', label='Baseline'),
        Patch(facecolor='tab:blue',   edgecolor='black', alpha=0.85, hatch='/', label='ZNE'),
        Patch(facecolor='tab:orange', edgecolor='black', alpha=0.85, hatch='x', label='PCE')
    ]
    ax.legend(
        handles=legend_elements,
        loc='best',
        framealpha=0.95,
        fontsize=20.7,
        borderpad=0.4,
        handlelength=1.0,
        handletextpad=0.6,
        labelspacing=0.5
    )

    # Bottom margin for two-line x labels
    plt.subplots_adjust(bottom=0.20)
    plt.tight_layout()

    # Save
    output_base = f"expectation_values_{num_qubits}q_{circ_depth}d"
    plt.savefig(f"{output_base}.pdf", bbox_inches='tight', pad_inches=0.1)
    plt.savefig(f"{output_base}.png", bbox_inches='tight', pad_inches=0.1, dpi=300)
    plt.show()

    print(f"\nFigure saved as {output_base}.pdf and {output_base}.png")
    print(f"\nIdeal: {ideal_mean:.6f}, Baseline: {baseline_mean:.6f}")
    print("\nMethod performance (mean ± std dev):")
    for label, mean_val, std_val, n in zip(labels[1:], means[1:], stds[1:], n_samples[1:]):
        clean_label = label.replace('\n', ' ')
        error = abs(mean_val - ideal_mean)
        improvement = (1 - error/abs(baseline_mean - ideal_mean)) * 100 if baseline_mean != ideal_mean else 0.0
        print(f"  {clean_label}:")
        print(f"    Mean: {mean_val:.6f} ± {std_val:.6f} (n={n} circuits)")
        print(f"    Error reduction: {improvement:.1f}%")

# Main execution
plot_expectation_values(results_file, num_qubits, circ_depth)

Loaded 5 circuit results



Figure saved as expectation_values_4q_25d.pdf and expectation_values_4q_25d.png

Ideal: 1.000000, Baseline: 0.634536

Method performance (mean ± std dev):
  Linear:
    Mean: 0.678925 ± 0.034154 (n=5 circuits)
    Error reduction: 12.1%
  Richardson:
    Mean: 0.819592 ± 0.032011 (n=5 circuits)
    Error reduction: 50.6%
  Linear  (2 checks):
    Mean: 0.991656 ± 0.050090 (n=5 circuits)
    Error reduction: 97.7%
  Exponential  (3 checks):
    Mean: 0.929920 ± 0.041467 (n=5 circuits)
    Error reduction: 80.8%
  Linear  (3 checks):
    Mean: 0.945542 ± 0.028023 (n=5 circuits)
    Error reduction: 85.1%
